In [ ]:
# TODO: replace df with dataframe
# TODO: investigate presence of multiple 'Plate ID' columns in condition_summary
# TODO: add dataset selector (or make new notebooks for analyzing different datasets - this might be better, think about it.)

In [ ]:
execfile('./src/display_utils.py')
execfile('./src/moldev_utils.py')
%matplotlib inline

In [ ]:
# cell_path = '../moldev-data/analyzed/06-24-2015/ssC LNA.csv'
# well_path = '../moldev-data/analyzed/06-24-2015/ssC LNA-wells.csv'
# condition_path = '../moldev-data/analyzed/06-24-2015/ssC LNA-conditions.csv'

cell_path = '../moldev-data/analyzed/06-24-2015/ssC PNA.csv'
well_path = '../moldev-data/analyzed/06-24-2015/ssC PNA-wells.csv'
condition_path = '../moldev-data/analyzed/06-24-2015/ssC PNA-conditions.csv'

colors = {'Blue & Red':'bwr','Blue':'Blues'}

In [ ]:
well_data = pd.read_csv(well_path)

condition_data = pd.read_csv(condition_path)

raw_cell_data = pd.read_csv(cell_path)
# cell_data = add_condition_cols(raw_cell_data)

---
# Well Plate Results

In [ ]:
interact(plot_plates, 
         dataframe = fixed(well_data),
         parameter = widgets.Select(value = 'Cell Count',
                                    options= get_params(well_data),
                                    height=250),
         function = well_data['Function'].unique().tolist(),
         color = colors,
         show = widgets.ToggleButtons(options=['None','Values','Conditions']));

---
# How do conditions cluster together?

In [ ]:
data = pipe(condition_data,
            get_means,
            lambda x: x.set_index('Condition'),
            lambda x: pd.DataFrame.drop(x,['Cell Type','Concentration'],axis=1),
            normalize_columns(fillna=True))
data_without_empty_params = data.dropna(axis=1,how='all')

In [ ]:
params_without_data = sorted(set(data.columns).difference(data_without_empty_params.columns))

In [ ]:
interact(matrix_tree, data = fixed(data_without_empty_params), color = colors);

---
# Heatmap subsets
The last selection box allows you to show parameters that are missing data. They shouldn't really affect clustering.

In [ ]:
def wrapper(conditions,features,color):
    df = data.loc[list(conditions)][list(features)]
    try: 
        matrix_tree(df,color)
    except:
        print "\n".join(["-"*50, "Please select multiple features and conditions.","-"*50])

subset = interactive(wrapper,
                     conditions = widgets.SelectMultiple(options = data_without_empty_params.index.values.tolist(), height = 300),
                     features = widgets.SelectMultiple(options = data_without_empty_params.columns.values.tolist(), height = 300),
                     color = colors)
subset

---
# How do parameters vary with drug dose?

In [ ]:
# dose_data = thread_first(condition_data,
#                         lambda df: df[df['Function'] == 'avg'],
#                         add_condition_cols)

# dose_error = thread_first(condition_data,
#                         lambda df: df[df['Function'] == 'sem'],
#                         add_condition_cols)

In [ ]:
# interact(dose_plot,
#          scale = widgets.ToggleButtons(options=['linear','log']), 
#          df = fixed(dose_data),
#          err = fixed(dose_error),
#          cols = widgets.SelectMultiple(value = ['Cell Count'],
#                               options= [col for col in dose_data.columns.values.tolist() if col not in ['Condition','Function','Base','Dose','Unit','Drug']],
#                               height=250));

---
#How are different parameters related?

In [ ]:
from pandas.tools.plotting import scatter_matrix

def scatter(df,cols):
    scatter_matrix(data[list(cols)], alpha=1, figsize=(20, 20), diagonal='kde');
    
interact(scatter, 
         df = fixed(data),
         cols = widgets.SelectMultiple(value = ['Cell Count'],
                              options= get_params(data),
                              height=250));

---
# Parameter vs. Conditions

In [ ]:
def plot_dist(data,parameters):        
    fig = plt.figure(num=None, figsize=(15, 7), dpi=100, facecolor='w', edgecolor='k')
    
    reshaped_data = pd.concat([add_col(data[['Condition',parameter]].rename(columns={parameter: 'Value'}),'Parameter',parameter) for parameter in parameters])
    
    sns.boxplot(x = 'Condition',
                y = 'Value',
                hue = 'Parameter',
                data = reshaped_data,
                order = sorted(data.Condition.unique().tolist(),key = sort_func))    
    
#     plt.ylabel(parameter,fontsize=20)
    plt.xlabel('Condition',fontsize=20)
    plt.title('Distribution: ' + parameter,fontsize=20)
    plt.tick_params(labelsize=10)
    plt.xticks(rotation=30,ha='right')
    mean = data[parameter].mean()
    plt.gca().set_ylim(bottom=data[parameter].mean() * -0.5)

In [ ]:
def dist_wrapper(parameters,conditions):
    conditions = list(conditions)
    if list(parameters) == ['Cell Count']:
        dataframe = thread_first(well_data,
                                 (filter_rows,'Function','avg'),
                                 (filter_rows,'Condition',conditions))
        plot_dist(dataframe,parameters)
    else: 
        if 'Cell Count' in parameters:
            print "Ignoring cell count. Select it by itself if you want to see it."
        dataframe = filter_rows(raw_cell_data,'Condition',conditions)
        plot_dist(dataframe,filter(lambda x: x != 'Cell Count',parameters))

In [ ]:
def sort_func(x):
    xs = x.split(' ')
    return (xs[-1],float(xs[-3]))

In [ ]:
interactive(dist_wrapper,
            parameters = widgets.SelectMultiple(options = get_params(raw_cell_data) + ['Cell Count'],height = 300),
            conditions = widgets.SelectMultiple(options = sorted(raw_cell_data.Condition.unique().tolist(),key = sort_func),height = 300))

---
# Compare conditions

In [ ]:
def compare_condition_dists(df, conditions, parameters, bandwidth=0.1, shaded = True, clip_ends = False, scale='linear'):
    n_rows = int(np.ceil(len(parameters)/3.0))
    plt.figure(figsize=(20,4 * n_rows))
    subs = gridspec.GridSpec(n_rows, 3) 
    plt.subplots_adjust(hspace=0.54,wspace=0.27)

    for parameter,sub in zip(parameters,subs):
        plt.subplot(sub)
        param_bandwidth = bandwidth * df[df.Condition == conditions[0]][parameter].mean()
        for condition in conditions:
            data = df[df.Condition == condition][parameter]
            kws = {"shade": shaded, "bw": param_bandwidth}
            if clip_ends: 
                kws = assoc(kws,"cut",0)
            sns.distplot(data, hist=False, kde_kws=kws,label=condition)
            plt.legend(loc = 0,fontsize=8)
        if scale == 'linear':
            plt.xlim(left=0)
        else:
            plt.xscale('log')
        plt.ylabel('Frequency')

In [ ]:
try:    
    interact(compare_condition_dists,
             df = fixed(raw_cell_data),
             conditions = widgets.SelectMultiple(options = raw_cell_data.Condition.unique().tolist(),height = 300),
             parameters = widgets.SelectMultiple(options = get_params(raw_cell_data),height = 300),
             bandwidth = (0,0.2,0.01),
             scale = widgets.ToggleButtons(options=['linear','log']));
except ValueError: 
    raise Exception("Hmm. That didn't work. Maybe one of the conditions or parameters you selected doesn't have any data.")

---
# Compare parameters

In [ ]:
def compare_parameter_dists(df, conditions, parameters, bandwidth=0.1, shaded = True, clip_ends = False, scale='linear'):
    n_rows = int(np.ceil(len(conditions)/3.0))
    plt.figure(figsize=(20,4 * n_rows))
    subs = gridspec.GridSpec(n_rows, 3) 
    plt.subplots_adjust(hspace=0.54,wspace=0.27)

    for condition,sub in zip(conditions,subs):
        plt.subplot(sub)
        condition_bandwidth = bandwidth * df[df.Condition == condition][parameters[0]].mean()
        for parameter in parameters:
            data = df[df.Condition == condition][parameter]
            kws = {"shade": shaded, "bw": condition_bandwidth}
            if clip_ends: 
                kws = assoc(kws,"cut",0)
            sns.distplot(data, hist=False, kde_kws=kws,label=parameter)
            plt.legend(loc = 0,fontsize=8)
        if scale == 'linear':
            plt.xlim(left=0)
        else:
            plt.xscale('log')
        plt.ylabel('Frequency')
        plt.xlabel(condition)

In [ ]:
interact(compare_parameter_dists,
         df = fixed(raw_cell_data),
         conditions = widgets.SelectMultiple(options = raw_cell_data.Condition.unique().tolist(),height = 300),
         parameters = widgets.SelectMultiple(options = get_params(raw_cell_data),height = 300),
         bandwidth = (0,0.2,0.01),
         scale = widgets.ToggleButtons(options=['linear','log']));